In [1]:
import os
from datetime import datetime, timedelta 
import time
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from scipy.signal import argrelextrema
from scipy.special import ndtr
from arch import arch_model

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, TimeSeriesSplit
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    make_scorer, accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, 
    roc_curve, RocCurveDisplay, auc, roc_auc_score, precision_recall_curve, PrecisionRecallDisplay, precision_score, recall_score, 
)
from sklearn.utils.class_weight import compute_class_weight
from xgboost import XGBClassifier, plot_importance

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import ReduceLROnPlateau

# import phik
# from phik.report import plot_correlation_matrix
# import fAux
# import mc_permutation
# import detrendPrice 


import ta
from ta.trend import EMAIndicator 
global global_returns
global global_labels

class FeatureDropper(BaseEstimator, TransformerMixin):
    def __init__(self, drop_cols):
        self.drop_cols = drop_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.drop(columns=self.drop_cols)
    


# ---

# GET DATA

def get_data(ticker, start_date='1999-01-01', end_date=None, save_csv=False):
    os.makedirs("data", exist_ok=True)
    filename = f"data/{ticker.lower()}.csv"
    # if os.path.exists(filename):
    #     print(f"Data for {i} already exists, skipping download.")
    #     continue

    data = yf.download(ticker.upper(), start=start_date, end=end_date, auto_adjust=True)

    # Handle empty or failed download
    if data.empty:
        print(f"No data found for {ticker}")
        return pd.DataFrame()
    
    # Flatten column headers if it's a MultiIndex (e.g., from group_by='ticker')
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)
    
    data['Ticker'] = ticker.lower()
    data.index = pd.to_datetime(data.index)
    data.columns = [col.lower() for col in data.columns]


    if save_csv:
        data.to_csv(filename, index=False)
        print(f"Saved data for {ticker} to {filename}")
        
    return data



# ---

# MODEL BUILDING

# model for longing; trading at open tomorrow (t+1)
# based on today's data (ohlcv and more), decide whether to long tomorrow
# binary target: 1 if price reverts up (above the threshold e.g. gain over 2%) wihtin the next five trading days, 0 otherwise
def create_target_long(df, threshold=0.02, lookahead=5):
    """
    Label as 1 if max return over the next 'lookahead' days is >= threshold.
    Else, label as 0.
    """
    # df = df.reset_index().rename(columns={'Date': 'date'})
    df['price_tmrw'] = df['open'].shift(-1) # trade at open next day

    df['seq_index'] = range(len(df))
    future_max_list = []
    for i in df['seq_index']:
        if i + 1 + lookahead <= len(df):
            window = df['open'].iloc[i+1 : i+1+lookahead]
            future_max_list.append(window.max())
        else:
            future_max_list.append(np.nan)

    df['future_max'] = future_max_list

    df['future_return'] = (df['future_max'] - df['price_tmrw']) / df['price_tmrw']
    df['target_long'] = (df['future_return'] >= threshold).astype(int)
    df.drop(columns=['seq_index', 'price_tmrw', 'future_max', 'future_return'], inplace=True)
    return df



def feature_engineering(df): # for trading at open tomorrow
    # original_feats = df.columns

    # phase 1: add features thats fundamental to mean reversion trading with bollinger bands and rsi
    df['sma30'] = df['close'].rolling(30).mean()
    df['sma10'] = df['close'].rolling(10).mean()
    df['sma_diff'] = df['sma10'] - df['sma30']
    df['sma_slope'] = df['sma10'].diff()
    df['std30'] = df['close'].rolling(30).std()
    df['bollinger_upper'] = df['sma30'] + 2 * df['std30']
    df['bollinger_lower'] = df['sma30'] - 2 * df['std30']
    df['percent_b'] = (df['close'] - df['bollinger_lower']) / (df['bollinger_upper'] - df['bollinger_lower'])
    df['bollinger_z'] = (df['close'] - df['sma30']) / df['std30']
    df['price_near_lower_bb'] = (df['close'] <= df['bollinger_lower'] * 1.01).astype(int)

    df['rsi14'] = ta.momentum.RSIIndicator(df['close'], window=14).rsi()    
    df['prod_bollingerz_rsi'] = df['percent_b'] * df['rsi14']

    df['rsi_smooth'] = df['rsi14'].rolling(3).mean()
    # Detect local lows
    rsi_vals = df['rsi_smooth'].values
    local_lows = argrelextrema(rsi_vals, np.less, order=5)[0]

    df['rsi_local_low'] = 0
    df.iloc[local_lows, df.columns.get_loc('rsi_local_low')] = 1

    # phase 2: add other technical features  
    df['daily_return'] = df['open'].pct_change()
    df['rolling_volatility14'] = df['daily_return'].rolling(window=30).std()
    df['atr'] = ta.volatility.AverageTrueRange(high=df['high'], low=df['low'], close=df['close']).average_true_range()

    # spy_ret5 (discard for now; if wanting to add add it from feature_engineering_add_macro)
    # spy_ret5 = df_spy['close'].pct_change(periods=5)
    # df = df.merge(spy_ret5.rename('spy_ret5'), left_index=True, right_index=True, how='left')

    # phase 3: more features!
    
    # 3.1 GARCH(1,1) on returns
    returns = df['close'].pct_change().dropna() * 100 # in percent
    am = arch_model(returns, vol='Garch', p=1, q=1)
    res = am.fit(disp='off')
    df['garch_vol'] = res.conditional_volatility # in percent

    # 3.2 tiem related features
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['week'] = df.index.isocalendar().week
    df['dayofweek'] = df.index.dayofweek

    # df.dropna(inplace=True)
    # new_feats = [i for i in df.columns if i not in original_feats]
    return df



def feature_engineering_add_macro(df,etf:str):
    df_etf = get_data(etf, end_date = df.index.max())
    df_etf['sma10'] = df_etf['close'].rolling(10).mean()
    df_etf['sma20'] = df_etf['close'].rolling(20).mean()
    df_etf['sma50'] = df_etf['close'].rolling(50).mean()
    df_etf['sma200'] = df_etf['close'].rolling(200).mean()
    df_etf['trend_10_50'] = (df_etf['sma10'] > df_etf['sma50']).astype(int)
    df_etf['trend_20_50'] = (df_etf['sma20'] > df_etf['sma50']).astype(int)
    df_etf['trend_50_200'] = (df_etf['sma50'] > df_etf['sma200']).astype(int)
    df = df.merge(df_etf['trend_10_50'].rename(f'{etf}_trend_10_50'), left_index=True, right_index=True, how='left')
    df = df.merge(df_etf['trend_20_50'].rename(f'{etf}_trend_20_50'), left_index=True, right_index=True, how='left')
    df = df.merge(df_etf['trend_50_200'].rename(f'{etf}_trend_50_200'), left_index=True, right_index=True, how='left')
    return df



def plot_model_metrics(y_true, y_proba):
    y_pred = (y_proba >= 0.5).astype(int)
    fig, axes = plt.subplots(1, 3, figsize=(14, 4))

    # 1. Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(ax=axes[0], colorbar=False)

    # 2. ROC Curve
    fpr, tpr, _ = roc_curve(y_true, y_proba)
    roc_auc = auc(fpr, tpr)
    roc_disp = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc)
    roc_disp.plot(ax=axes[1])

    # 3. Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_true, y_proba)
    pr_disp = PrecisionRecallDisplay(precision=precision, recall=recall)
    pr_disp.plot(ax=axes[2])

    plt.tight_layout()
    plt.show()



def eval_pipe(X, y, pipe, n_split=5, predict_proba_threshold=0.5):
    '''
    for evaluations on train set, make sure X and y does not contain anything from the test set (future data), 
    and use n_split=5 for 5-fold time series cross-validation
    '''
    tscv = TimeSeriesSplit(n_splits=n_split)
    for i, (train_index, test_index) in enumerate(tscv.split(X)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        pipe.fit(X_train, y_train)
        y_proba = pipe.predict_proba(X_test)[:, 1]
        y_pred = (y_proba >= predict_proba_threshold).astype(int)

        print(f'Fold {i+1}:\n{classification_report(y_test, y_pred)}')

        if i == 0:
            print('Fold 1 visualized below')
            plot_model_metrics(y_test, y_proba)

def pred_proba_to_signal(y_proba, threshold=0.5):
    """
    Convert predicted probabilities to binary signals.
    1 if probability >= threshold, else 0.
    returns a Series of signals.
    """
    return (y_proba >= threshold).astype(int)


def entry_exit(df, use_vol=None, take_profit=1, stop_loss=1): 
    '''
    FOR EACH INSTRUMENT (don't use the aggregated one that contains multiple tickers)
    takes in the X_test or ohlcv dataframe containing model signals,
    returns a df that contains entry and exit dates, prices, returns, and holding days
    use_vol: either 'atr' or 'garch_vol'
    take_profit and stop loss as percentage
    '''
    # if date as index
    if pd.api.types.is_datetime64_any_dtype(df.index): 
        df = df.reset_index().rename(columns={'Date': 'date'})

    trades = []
    i = 0
    n = len(df)
    if use_vol not in [None, 'atr', 'garch_vol']:
        raise ValueError("use_vol must be one of: None, 'atr' or 'garch_vol'")
    
    if use_vol == 'atr':
        multiplier = df['atr']
    elif use_vol == 'garch_vol':
        multiplier = df['garch_vol']
    else:
        multiplier = pd.Series(1, index=df.index)

    while i < n - 6:  # we need at least 5 days ahead, plus trading at open tomorrow
        if df['model_signal'].iloc[i] == 1:
            entry_date = df['date'].iloc[i+1]
            entry_price = df['open'].iloc[i+1]
            exit_price = None
            exit_date = None
            holding = None
            exit_reason = None

            for j in range(1, 6):  # check up to 5 days ahead
                if i + 1 + j >= n:
                    break

                next_price = df['open'].iloc[i + 1 + j]
                ret = (next_price - entry_price) / entry_price

                # Exit Conditions
                if ret >= take_profit * 0.01 * multiplier.iloc[i]:  # profit target
                    exit_price = next_price
                    exit_date = df['date'].iloc[i + 1 + j]
                    holding = j
                    exit_reason = 'profit_target'
                    break
                elif ret <= -stop_loss * 0.01 * multiplier.iloc[i]:  # stop loss
                    exit_price = next_price
                    exit_date = df['date'].iloc[i + 1 + j]
                    holding = j
                    exit_reason = 'stop_loss'
                    break
                elif df['open'].iloc[i + 1 + j] >= df['sma30'].iloc[i + 1 + j]:  # revert to SMA30
                    exit_price = next_price
                    exit_date = df['date'].iloc[i + 1 + j]
                    holding = j
                    exit_reason = 'revert_to_sma30'
                    break

            if exit_price is None:
                # Max holding (5th day)
                exit_price = df['open'].iloc[i + 1 + 5]
                exit_date = df['date'].iloc[i + 1 + 5]
                exit_reason = 'max_holding_expired'
                holding = 5

            trade_return = (exit_price - entry_price) / entry_price
            trades.append({
                'entry_date': entry_date,
                'exit_date': exit_date,
                'entry_price': entry_price,
                'exit_price': exit_price,
                'return': trade_return,
                'holding_days': holding,
                'exit_reason': exit_reason
            })

            i = i + holding  # skip to the day after exit
        else:
            i += 1

    if len(trades) > 0:
        return pd.DataFrame(trades)
    
    return pd.DataFrame(columns=[
        'entry_date',
        'exit_date',
        'entry_price',
        'exit_price',
        'return',
        'holding_days',
        'exit_reason'
    ])



def backtest(X_test, model, proba_threshold = 0.5, use_vol=None, take_profit=1, stop_loss=1):
    trade_stats = {} 

    y_proba = model.predict_proba(X_test)[:, 1]
    trade_stats['max_proba'] = y_proba.max()

    signals = pred_proba_to_signal(y_proba, threshold=proba_threshold)
    X_test_signal = X_test.assign(model_signal=signals, y_proba=y_proba)

    df_trade = entry_exit(X_test_signal, use_vol, take_profit, stop_loss)
    if len(df_trade) > 0:
        trade_stats['exit_reason_spread'] = df_trade['exit_reason'].value_counts()
        trade_stats['holding_days_spread'] = df_trade['holding_days'].value_counts()

        total_holding_days = df_trade['holding_days'].sum()
        total_trading_days = len(X_test_signal)
        if total_trading_days > 0:
            holding_time_percentage = total_holding_days/total_trading_days
        else: 
            holding_time_percentage = 0

        n_trades = len(df_trade)
        n_wins = len(df_trade[df_trade['return'] > 0])
        if n_trades > 0:
            win_rate = n_wins/n_trades
        else: 
            win_rate = 0

        trade_stats['total_holding_days'] = total_holding_days
        trade_stats['total_trading_days'] = total_trading_days
        trade_stats['holding_time_percentage'] = holding_time_percentage
        trade_stats['n_trades'] = n_trades
        trade_stats['n_wins'] = n_wins
        trade_stats['win_rate'] = win_rate
    
    else:
        trade_stats['exit_reason_spread'] = 'not available'
        trade_stats['holding_days_spread'] = 'not available'
        trade_stats['total_holding_days'] = 0
        trade_stats['total_trading_days'] = 0
        trade_stats['holding_time_percentage'] = 0
        trade_stats['n_trades'] = 0
        trade_stats['n_wins'] = 0
        trade_stats['win_rate'] = 0

    return df_trade, trade_stats



def get_cagr(df):
    if len(df) > 0:
        start_date = df['entry_date'].min()
        end_date = df['exit_date'].max()
        n_years = (end_date - start_date).days / 365.25
        capital = 1
        for r in df['return']:
            capital *= (1 + r)
        cagr = capital ** (1 / n_years) - 1
        return cagr
    return 0

def get_sharpe(df_trades, total_trading_days):
    if len(df_trades) > 0 and total_trading_days > 0:
        ret_mean = df_trades['return'].mean()
        ret_std = df_trades['return'].std()
        n_trades = len(df_trades)
        trades_per_year = (n_trades/total_trading_days) * 252
        return (ret_mean/ret_std) * np.sqrt(trades_per_year)
    return 0


def check_return_stats(df):
    if len(df) > 0:
        print(f"check return stats: \n {df['return'].describe()}")
        sns.histplot(df['return'])
    print('no trade available')


def safe_format(value, fmt, default="N/A"):
    try:
        return format(value, fmt) if value is not None else default
    except (TypeError, ValueError):
        return default

def print_test_set_result(ticker, dfs):
    result = dfs[ticker]['test_set_result']
    
    print(
        f"ticker: {ticker} \n"
        f"total_holding_days: {result.get('total_holding_days', 'N/A')} \n"
        f"total_trading_days: {result.get('total_trading_days', 'N/A')} \n"
        f"holding_time_percentage: {safe_format(result.get('holding_time_percentage'), '.2%')} \n" 
        f"n_trades: {result.get('n_trades', 'N/A')} \n" 
        f"win_rate: {safe_format(result.get('win_rate'), '.2%')} \n" 
        f"cagr: {safe_format(result.get('cagr'), '.2%')} \n" 
        f"sharpe: {safe_format(result.get('sharpe'), '.3')} \n" 
        "---"
    )

# ---

# FOR DAILY OPERATION AND AUTOMATION (a complete set will be in the py script)

# def time_to_retrain(train_date_str, retrain_freq_days=7):
#     train_date = datetime.fromisoformat(train_date_str).date()
#     today = datetime.today().date()
#     return (today - train_date).days >= retrain_freq_days

In [2]:
# first 25 stocks are the top 25 holdings in qqq as of jul 17 2025
# plus a couple others from my watchlist
dfs_raw = {}
stocks = [
    'nvda','msft','aapl','amzn','avgo',
    'meta','nflx','tsla','googl','cost',
    'goog','pltr','csco','amd','tmus',
    'lin','intu','pep','txn','isrg',
    'bkng','qcom','amgn','adbe','amat',
    'spot','tsm','rddt','mstr','rklb',
]

for i in stocks:
    try: 
        df = get_data(i, end_date='2025-07-17')
        if not df.empty:
            dfs_raw[i] = df
        else:
            print(f"{i} returned empty data.")
    except Exception as e:
        print(f"{i} failed with: {e}")
    time.sleep(1)  

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [3]:
dfs = {}
for key, value in dfs_raw.items():
    dfs[key] = {}
    this_df = value.copy()
    dfs[key]['ohlcv'] = this_df
    this_df = create_target_long(this_df)
    this_df = feature_engineering(this_df)
    this_df = feature_engineering_add_macro(this_df, 'spy')
    this_df = feature_engineering_add_macro(this_df, 'qqq')
    this_df.dropna(inplace=True)
    X = this_df.drop(columns=['target_long','ticker'])
    y = this_df['target_long']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
    # dfs[key]['X'] = X
    # dfs[key]['y'] = y
    dfs[key]['X_train'] = X_train
    dfs[key]['X_test'] = X_test
    dfs[key]['y_train'] = y_train
    dfs[key]['y_test'] = y_test

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [4]:
drop_feats = [
    # 'date',
    'high','low','sma30','sma10','std30',
    'bollinger_upper','bollinger_lower',
    'rsi_smooth',
]

ct = make_column_transformer(
    ('drop', drop_feats),  
    remainder='passthrough'  
)

pipe = Pipeline([
    ('ct', ct),
    ('xgb', XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        use_label_encoder=False,
        random_state=42
    ))
])

param_grid = {
    'xgb__max_depth': [3, 5, 7],
    'xgb__learning_rate': [0.01, 0.05, 0.1],
    'xgb__n_estimators': [100, 300, 500],
    'xgb__subsample': [0.5, 0.7, 1.0],
    'xgb__colsample_bytree': [0.5, 0.7, 1.0]
}


# smaller param grid

# param_grid = {
    # 'xgb__max_depth': [3, 5],                  # shallower trees generalize better
    # 'xgb__learning_rate': [0.05, 0.1],         # fast vs. slow learner
    # 'xgb__n_estimators': [100, 300],           # fewer rounds is faster
    # 'xgb__subsample': [0.7, 1.0],              # some regularization
    # 'xgb__colsample_bytree': [0.7, 1.0]        # helps reduce overfitting
# }


rscv = RandomizedSearchCV(
    pipe,
    param_distributions=param_grid,
    n_iter=30,  # number of random combinations to try
    scoring='f1',  # or 'roc_auc', 'accuracy', etc.
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

In [5]:
for key in dfs.keys():
    # train set model
    this_rscv = clone(rscv)
    this_rscv.fit(dfs[key]['X_train'], dfs[key]['y_train'])
    dfs[key]['model'] = this_rscv
    dfs[key]['train_set_model'] = {
        'best_estimator': this_rscv.best_estimator_,
        'best_params': this_rscv.best_params_,
        'best_score': this_rscv.best_score_,
        'train_date': datetime.today().date().isoformat()  # e.g., '2025-07-24'
    }

    # test set result
    # backtest with proba_threshold = 0.6, garch_vol as multiplier, 1.5 to 1 risk reward ratio
    df_trade, trade_stats = backtest(
        dfs[key]['X_test'], 
        this_rscv, 
        proba_threshold=0.6, 
        use_vol = 'garch_vol',
        take_profit=1.5,
        stop_loss=1
    )

    dfs[key]['test_set_result'] = {
        'proba_threshold': 0.6,
        'use_vol': 'garch_vol',
        'take_profit': 1.5,
        'stop_loss': 1
    }

    dfs[key]['test_set_result']['df_trade'] = df_trade
    for k,v in trade_stats.items():
        dfs[key]['test_set_result'][k] = v

    dfs[key]['test_set_result']['cagr'] = get_cagr(df_trade)
    
    dfs[key]['test_set_result']['sharpe'] = get_sharpe(
        df_trade, 
        dfs[key]['test_set_result']['total_trading_days']
    )

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:54:40] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:54:40] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:54:40] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:54:40] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:54:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:54:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:54:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:54:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not

Fitting 5 folds for each of 30 candidates, totalling 150 fits


...

/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:59:16] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:59:16] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:59:16] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:59:16] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:59:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:59:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:59:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:59:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:59:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:59:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:59:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/stevey/cpsc330arm/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:59:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not

In [6]:
for idx, i in enumerate(list(dfs.keys())):
    print(f"{i}", end='  ')
    if idx % 5 == 4:
        print()  # new line after every 5 keys

nvda  msft  aapl  amzn  avgo  
meta  nflx  tsla  googl  cost  
goog  pltr  csco  amd  tmus  
lin  intu  pep  txn  isrg  
bkng  qcom  amgn  adbe  amat  
spot  tsm  rddt  mstr  rklb  


In [7]:
print_test_set_result('nvda', dfs)
print_test_set_result('spot', dfs)
print_test_set_result('pltr', dfs)
print_test_set_result('tsla', dfs)
print_test_set_result('tsm', dfs)
print_test_set_result('tmus', dfs)
print_test_set_result('rklb', dfs)
print_test_set_result('mstr', dfs)
print_test_set_result('rddt', dfs)


ticker: nvda 
total_holding_days: 140 
total_trading_days: 1326 
holding_time_percentage: 10.56% 
n_trades: 78 
win_rate: 82.05% 
cagr: 69.42% 
sharpe: 3.56 
---
ticker: spot 
total_holding_days: 56 
total_trading_days: 361 
holding_time_percentage: 15.51% 
n_trades: 45 
win_rate: 68.89% 
cagr: 65.60% 
sharpe: 2.25 
---
ticker: pltr 
total_holding_days: 13 
total_trading_days: 235 
holding_time_percentage: 5.53% 
n_trades: 11 
win_rate: 81.82% 
cagr: 65.82% 
sharpe: 2.91 
---
ticker: tsla 
total_holding_days: 139 
total_trading_days: 751 
holding_time_percentage: 18.51% 
n_trades: 84 
win_rate: 65.48% 
cagr: 88.16% 
sharpe: 2.1 
---
ticker: tsm 
total_holding_days: 159 
total_trading_days: 1326 
holding_time_percentage: 11.99% 
n_trades: 78 
win_rate: 69.23% 
cagr: 20.64% 
sharpe: 1.28 
---
ticker: tmus 
total_holding_days: 27 
total_trading_days: 912 
holding_time_percentage: 2.96% 
n_trades: 10 
win_rate: 80.00% 
cagr: 4.09% 
sharpe: 0.835 
---
ticker: rklb 
total_holding_days: 3 
to

In [8]:
def print_dict_structure(d, indent=0):
    indent_str = '  ' * indent
    if not isinstance(d, dict):
        print(f"{indent_str}- {type(d).__name__}")
        return

    for key, value in d.items():
        print(f"{indent_str}- {key}: {type(value).__name__}")
        if isinstance(value, dict):
            print_dict_structure(value, indent + 1)

print_dict_structure(dfs['nvda'])



- ohlcv: DataFrame
- X_train: DataFrame
- X_test: DataFrame
- y_train: Series
- y_test: Series
- model: RandomizedSearchCV
- train_set_model: dict
  - best_estimator: Pipeline
  - best_params: dict
    - xgb__subsample: float
    - xgb__n_estimators: int
    - xgb__max_depth: int
    - xgb__learning_rate: float
    - xgb__colsample_bytree: float
  - best_score: float64
  - train_date: str
- test_set_result: dict
  - proba_threshold: float
  - use_vol: str
  - take_profit: float
  - stop_loss: int
  - df_trade: DataFrame
  - max_proba: float32
  - exit_reason_spread: Series
  - holding_days_spread: Series
  - total_holding_days: int64
  - total_trading_days: int
  - holding_time_percentage: float64
  - n_trades: int
  - n_wins: int
  - win_rate: float
  - cagr: float
  - sharpe: float64
